In [1]:
import cv2
import numpy as np
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from numpy import argmax
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import copy
import os
from google.colab.patches import cv2_imshow

In [45]:
from scipy.ndimage import affine_transform
from scipy.ndimage import interpolation

In [9]:
model = load_model('/content/drive/MyDrive/model_fruit/bao/model_6.h5')

In [ ]:
#plt.imshow(sharpen_img, cmap='gray')
#plt.show()
#Hiển thị ảnh kết quả

In [47]:
def moments(image):
    c0, c1 = np.mgrid[:image.shape[0], :image.shape[1]]
    totalImage = np.sum(image)  # sum of pixels
    m0 = np.sum(c0 * image) / totalImage
    m1 = np.sum(c1 * image) / totalImage
    m00 = np.sum((c0 - m0) ** 2 * image) / totalImage
    m11 = np.sum((c1 - m1) ** 2 * image) / totalImage
    m01 = np.sum((c0 - m0) * (c1 - m1) * image) / totalImage
    mu_vector = np.array([m0, m1])
    covariance_matrix = np.array([[m00, m01], [m01, m11]])
    return mu_vector, covariance_matrix

def deskew(image):
    c, v = moments(image)
    alpha = v[0, 1] / v[0, 0]
    affine = np.array([[1, 0], [alpha, 1]])
    ocenter = np.array(image.shape) / 2.0
    offset = c - np.dot(affine, ocenter)
    return affine_transform(image, affine, offset=offset)

def remove_grid(binaryImage):
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
    closing = cv2.morphologyEx(binaryImage, cv2.MORPH_OPEN, kernel, iterations=2)

    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    closing = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernel2, iterations=1)
    return closing

In [50]:
def run_example(path):
  try:
        image = cv2.imread(path)

        inputImageCopy = image.copy()
        grayscaleImage = cv2.cvtColor(inputImageCopy, cv2.COLOR_BGR2GRAY)
        threshValue, binaryImage = cv2.threshold(grayscaleImage, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        binaryImage = remove_grid(binaryImage)
        img_resize1 = deskew(binaryImage)
        gray = cv2.resize(img_resize1, (28, 28), interpolation=cv2.INTER_AREA)

        sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])

        sharpen_img = cv2.filter2D(gray, -1, sharpen_kernel)

        sharpen_img = sharpen_img.astype('float32')
        sharpen_img = sharpen_img / 255.0


        #Hiển thị ảnh kết quả
        image_file = np.array(sharpen_img)
        prediction = model.predict(image_file.reshape(1, 28, 28, 1))
        digit = argmax(prediction)
        return digit
  except:
    return 0

In [51]:
path = "/content/drive/MyDrive/data/handtest/2c5256ec1f58ae30941329f07ff52e11  -.png"
run_example(path)

1/1 [==============================] - 0s 22ms/step


3

In [52]:
results = []
org_dir = "/content/drive/MyDrive/data/handtest"

for item in os.listdir(org_dir):
    dir1 = os.path.join(org_dir, item)
    digit = run_example(dir1)
    results.append([item, digit])

1/1 [==============================] - 0s 21ms/step


In [53]:
import pandas as pd


# Chuyển danh sách kết quả thành DataFrame
df = pd.DataFrame(results, columns=['file_name', 'label'])

# Lưu DataFrame thành file CSV
df.to_csv('output22.csv', index=False, header=False)